In [ ]:
import chargeback_rpt.db_postgres_command as db_command
import chargeback_rpt.vm_data_utility as vm_util
import chargeback_rpt.vm_data_validator as vx
import chargeback_rpt.email_notifier as x_mail
import chargeback_rpt.file_directory_manager as fd_mn
import pandas as pd
import numpy as np



import openpyxl

import psycopg2
import psycopg2.extras as extras

import datetime
import json
import os
import sys 



# For run script .py

In [ ]:

#datasource_key='nimble_vol'
datasource_key='primera_vol'

run_py=False
fix_mmyy=True
is_only_check_data=False

x_datenow=datetime.datetime.now()
month_x=x_datenow.strftime('%m')
year_x=x_datenow.strftime('%Y')

if fix_mmyy:
 month_param='10'
 year_param='2021'
else:
 month_param=month_x
 year_param=year_x
    
    
print(f"Current month-year : {month_param}-{year_param}")

if  run_py:

    press_y='n'
    ok=False
    
    if len(sys.argv) > 1 :
        if sys.argv[1]=='0'  and sys.argv[2]=='0'  and  (sys.argv[3]=='1' or sys.argv[3]=='2'):
         month_param=month_x
         year_param=year_x 
         if sys.argv[3]=='1':
            datasource_key='nimble_vol'
         elif sys.argv[3]=='2':
            datasource_key='primera_vol'   
         
         is_only_check_data=bool(int(sys.argv[4]))
            
         ok=True   
#          if ok:
#           f = open(f'test_run_ok_{datasource_key}.txt', 'a') 
#           f.write(f"ok run {datasource_key}")
#           f.close()
            
        else:
         print("The system are required 3 arguments as the following like this.")   
         print("0 0 1")  
         print("0 0 2")  
         raise Exception("Specify arguments orderly such as  0 as first , 0 as second ,nimble_vol or primera_vol as third ")   
    
    else:
        
        print("Enter HPE storage to import data.")
        print("1 is Nimble")
        print("2 is Primera")
        datasource_type =int(input("Enter storage model , press 1  or  2: "))
        if datasource_type in [1,2]:
            if datasource_type==1:
               datasource_key='nimble_vol'
            else:
               datasource_key='primera_vol'
        else:
             raise Exception("Invalid storage model , allow only press the following choices\n 1 = Nimble \n 2 = Primera")   
            
            
        
        print("Enter month and year . If you want current month and year.Press Enter ")
        month_param = int(input("Enter the month (1-12) : ") or month_x)
        year_param = int(input("Enter the year(such as 2021,2022,2023...) : ") or year_x)
        print("============================================================================")
        print(f"Do import {datasource_key.title()} for month={month_param} and year={year_param}")
    
        press_y=input(f"Press y=True and any keys=False : ") 
        if press_y.lower()=='y':
         ok=True


    if ok==True:
        try:
         report_dt= datetime.datetime(int(year_param), int(month_param), 1)
         print(report_dt)   
         month_param=report_dt.strftime('%m')
         year_param=report_dt.strftime('%Y')   

        except Exception as ex:
         print(ex)   
         raise ex
    else:
         quit()
            
              

In [ ]:
# check in job_type table

if is_only_check_data==False:
 job_type = 6 if  datasource_key=='nimble_vol' else 7 
else:
 job_type = 10 if  datasource_key=='nimble_vol' else 11
 

t_id=vm_util.creating_transaction(job_type,month_param,year_param)
print(f"ETL Transaction ID: {t_id}")

list_error=[]
print(list_error)

In [ ]:
if  datasource_key=='nimble_vol':
 comment_col='comment_nimble'
 covert_from_to_gb='byte'
 used_size_col='size_used_gb_nimble'   
 vol_master_col='volume_name_nimble'   
 job_type=6
 tb_key="nimble_table"
 hpe_col_key='nimble'   

elif datasource_key=='primera_vol':
 comment_col='comment_primera'
 covert_from_to_gb='mb'
 used_size_col='size_used_gb_primera'  
 vol_master_col='volume_name_primera'   
 job_type=7
 tb_key="primera_table"
 hpe_col_key='premira'   

path_key='import_path'

center_col='cost_center'
system_col='system_name'
created_date_col='created_date'

cs_cols_toCheck=[comment_col,center_col,system_col]

hpe_ok=False
vol_filepath=None



In [ ]:
# Move  to util
def check_error_point_etl(tran_id):
  print(list_error)
  if True in list_error:

        
    etl_files=[]
    if  vol_filepath is not None and os.path.exists(vol_filepath):
     etl_files.append([hpe_ok,vol_filepath])   
    
    if len(etl_files)>0 and (hpe_ok==False) :
     vm_util.finished_etl_folder(t_id,hpe_col_key,False,etl_files)  
        
    vm_util.collect_error_to_sent_mail(tran_id) 
    
    print("ETL occured error") 
    
    raise Exception("Program is teminated and check error from email and log_error.txt")
    
  list_error.clear()
  print(list_error)   

In [ ]:
try:
    datetime_format = vm_util.get_config_value("datetime_format", t_id)
    print(datetime_format)
    
except Exception as ex:
    list_error.append(True)
    
check_error_point_etl(t_id)  

In [ ]:
print("get root folder and source file name for ready to do ETL")

try: 
 
 # load root path
 import_path=vm_util.get_config_value(path_key,t_id)
 existPath,is_error=vx.check_existing_filepath(import_path,t_id)
 list_error.append(is_error)   
 if existPath:
   print("Get Path of all source files: ",import_path)   

 # load datasource data    
 sr_vol,is_error=vx.check_source_name_file(datasource_key,t_id)
 list_error.append(is_error) 
 print(sr_vol)
 
 # load datafsource file paht
 vol_filepath=f"{import_path}\\{sr_vol['source_prefix']}.{sr_vol['source_type']}"
 a,is_error=vx.check_existing_filepath(vol_filepath,t_id)
 list_error.append(is_error)    
 print("Get Path of volume file: ",vol_filepath,' is ',a)   

 # Select columns from source to load
 print("Selec required columns from source file for dataframe")   
 df_vol_mappingFields=vm_util.get_active_datafield(sr_vol['id'])
 print(df_vol_mappingFields[['column_table_name','field_source_name','is_not_null','is_cost_column']])
 
 vol_cols=df_vol_mappingFields['field_source_name'].tolist()   
 print(vol_cols)
    
 
 
 # load data field   


except Exception as ex:
 list_error.append(True)  
   
    
check_error_point_etl(t_id)  

In [ ]:
print("load volume file")

if os.path.isfile(vol_filepath) and os.path.getsize(vol_filepath) > 0:
 df_vol= pd.read_csv(vol_filepath,usecols=vol_cols)

 if df_vol.empty==False:
 
     print("Mapping external columnns from source file to  interal fields on system")
     df_vol,err=vx.map_fields_ds_to_cols_df(df_vol,df_vol_mappingFields,vol_filepath,t_id)

     print(df_vol.info())
     print(df_vol)
    
 else:
    
     if is_only_check_data==True:
          delete_resultList= fd_mn.delele_file(vol_filepath)
     else:
          vm_util.finished_etl_folder(t_id,hpe_col_key,True,[[True,vol_filepath]])    
            
     updated_rows=vm_util.created_transaction(t_id)
     print(f"completed ETL with No volumne on {hpe_col_key.title()} for  month={month_param} and year={year_param}")   
     exit()   
    
    
else:

  try:
    
     if is_only_check_data==True:
          delete_resultList= fd_mn.delele_file(vol_filepath)
     else:
          vm_util.finished_etl_folder(t_id,hpe_col_key,True,[[True,vol_filepath]]) 
        
     updated_rows=vm_util.created_transaction(t_id)
     print(f"completed ETL with No volumne on {hpe_col_key.title()} for  month={month_param} and year={year_param}")   
     exit()      
    
  except Exception as ex:
    list_error.append(True)
    print(ex)
    

#df_vol
check_error_point_etl(t_id)  

# Transform Data

In [ ]:
print("Extract and Transform Data")

In [ ]:
print("Extract Cost Center and System Name and Created Date")
df_vol[center_col]=df_vol[comment_col].apply(vm_util.split_comment_to_each_value,args=(center_col,)) 
df_vol[system_col]=df_vol[comment_col].apply(vm_util.split_comment_to_each_value,args=(system_col,))
df_vol[created_date_col]=df_vol[comment_col].apply(vm_util.split_comment_to_each_value,args=(created_date_col,))


#df_vol

In [ ]:
print("Check invalid  data after extracting comment")
cols_na_comment,df_na_comment=vx.find_NaN(df_vol,cs_cols_toCheck)

print(cols_na_comment)
print(df_na_comment)

if df_na_comment.shape[0]>0:
    
 str_error = '</br>Summarize rows that found any null or incorrect format string in <b><u>comment column</u></b></br>'
 df_na_comment= df_na_comment.fillna("")
 str_error += df_na_comment.to_html(index=False)
 vm_util.add_error_to_database(16, str_error,t_id)
 list_error.append(True)   


print(df_vol.info())
print(df_vol)



check_error_point_etl(t_id)  

# Convert string date to datetime

In [ ]:
import calendar
lastDay_val=datetime.date(int(year_param),int(month_param),calendar.monthrange(int(year_param), int(month_param))[1])
print(lastDay_val)
print("Validate CreatedDate")

df_vol=vx.find_incorrrect_format_for_input_datetime(df_vol,created_date_col,vol_master_col,datetime_format,t_id)
if df_vol is None:
 list_error.append(True)
 print(list_error)
else:    
 df_vol=vx.find_invalid_x_date_greater_last_date_for_input_datetime(df_vol,created_date_col,lastDay_val,vol_master_col,t_id)
 if df_vol is None:
  list_error.append(True) 
  print(list_error)
 else:
   print(f"all created_date > {lastDay_val}")   
   print( df_vol[[vol_master_col,created_date_col]] ) 
    
check_error_point_etl(t_id)

# Convert size unit 

In [ ]:
print("Convert size unit")
df_vol[used_size_col]=df_vol[used_size_col].apply(vm_util.convert_Xb_unit_Gb,args=(covert_from_to_gb,))
#df_vol  
check_error_point_etl(t_id)

In [ ]:
print(df_vol.info())
print(df_vol)
#df_vol

# Add Addtional Data

In [ ]:
print("Transform addtoinal data")
try:
     

 df_vol['month']=month_param
 df_vol['year']=year_param
 df_vol['import_date']=x_datenow


 df_vol['transaction_id']= t_id

 df_vol['system_name']=df_vol['system_name'].apply(vx.replace_x_character)
 df_vol['cost_center']=df_vol['cost_center'].apply(vx.replace_x_character)   


except Exception as err:
  list_error.append(True)
  error_message= str(err)
  print(error_message)   
  vm_util.add_error_to_database(14,error_message,t_id)  
 
check_error_point_etl(t_id)
hpe_ok=True

In [ ]:
print(df_vol.info())
print(df_vol)

# End up with  Completed ETL in only Checking Data mode

In [ ]:
if is_only_check_data==True:
    try: 

        delete_resultList= fd_mn.delele_file(vol_filepath)   
        updated_rows=vm_util.created_transaction(t_id)
        print("completed ETL in only Checking Data mode")
        
        exit()
        
        
    except Exception as ex:
        list_error.append(True)
        print(ex)
        raise Exception("Program is teminated and check error from email and log_error.txt")

# Save to database

In [ ]:
print("save  volumn dataframe to database")
table_name,listCols_report,is_error=vx.listCols_report_table(tb_key,t_id)
list_error.append(is_error)


print(f'list all columns in {table_name} table: ',len(listCols_report))
print(listCols_report)


listCols_dfx=df_vol.columns.tolist()
print(f'list all columns in {table_name} dataframe: ',len(listCols_dfx))
print(listCols_dfx)


check_columns_2=vm_util.verify_existing_columns(listCols_dfx,listCols_report)

print(check_columns_2)

In [ ]:
print("Save data to database")
if check_columns_2  is None:
   df_vol=df_vol[listCols_report]
   print(df_vol.info())
   print(df_vol.head())
   rslt=db_command.add_data_values(db_command.get_postgres_conn(),df_vol,table_name)
   #print(rslt)
    
else:
   list_error.append(True)
   error_message=check_columns_2
   vm_util.add_error_to_database(10,error_message,t_id)
   print(error_message)


In [ ]:
check_error_point_etl(t_id)

# Check New CC

In [ ]:
print("Update new cost center")

try :
 cost_centerList = df_vol['cost_center'].unique()
 df_new_cc=vm_util.add_new_cost_center(cost_centerList,f'{hpe_col_key.title()}-ETL',t_id)
 print(df_new_cc)
except Exception as ex:
 list_error.append(True)   
 print(ex)   

check_error_point_etl(t_id)


# Complete Transaction

In [ ]:
try: 
    etl_files=[[hpe_ok,vol_filepath]]
    if hpe_ok==True :
     vm_util.finished_etl_folder(t_id,hpe_col_key,True,etl_files)    
     updated_rows=vm_util.created_transaction(t_id)
     print("completed ETL")
    
except Exception as ex:
    list_error.append(True)
    print(ex)
    
check_error_point_etl(t_id)